In [1]:
pwd

u'/Users/chrisrytting1/vim/work/OSPC/C-TAM/Housing'

In [2]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import random
import statsmodels.discrete.discrete_model as sm
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier as rf


In [3]:
def Init_CPS_Vars(CPS):

    '''
        Variables within the CPS that we use:

    ssi_val     Supplemental Security income amount received
    ssi_yn      Supplemental Security income received
    ssikidyn    Supplemental Security income, child received
    rednssi1    Supplemental Security income, reason 1
    rednssi2    Supplemental Security income, reason 2
    marsupwt    March supplement final weight
    a_age       Age
    a_spouse    Marital Status
    fownu18     Non-married children under 18 in household
    gestfips    FIPS State Code
    pedisdrs    Disability, dressing or bathing
    pedisear    Disability, hearing
    pediseye    Disability, seeing
    pedisout    Disability, doctor visits, shopping alone
    pedisphy    Disability, walking, climbing stairs
    pedisrem    Disability, remembering
    dis_hp      Health problem or a disability which prevents working
    rsnnotw     Reason for not working
    vet_typ1    Veterans payments, type 1
    pemlr       Monthly labor force recode
    mcare       Medicare coverage
    wsal_val    Total wage and salary earnings value
    semp_val    Own business self-employment earnings, total value
    frse_val    Farm self-employment earnings, total value
    ss_val      Social Security payments received, value
    rtm_val     Retirement income received, total amount
    oi_off      Income sources, other
    oi_val      Income, other (amount)
    uc_yn       Unemployment compensation benefits received
    uc_val      Unemployment compensation benefits value
    int_yn      Interest received
    int_val     Interest income received, amount+
    ffpos       Record type and sequence indicator (similar numbers
            indicate in same family)
    fh_seq      Household sequence number (similar number indicate same Household)
    finc_ssi    Supplemental Security benefits (Family)
    ftot_r      Total family income
    ftotval     Total family income
    ptot_r      Person income, total
    ptotval     Person income, total
    peridnum    Unique Person identifier
    '''
    # These lists contain the column names that contain the missing data
    Noneornot = ['ssi_val', 'csp_val', 'rnt_val', 'div_val', 'vet_val', 'wsal_val', 'semp_val', 'frse_val', 'ss_val', 'rtm_val', 'oi_val', 'uc_val', 'int_val', 'ftotval', 'ptotval', 'hwsval', 'pearnval', 'htotval']
    NotInUniv = ['ssi_yn', 'dis_hp', 'rsnnotw', 'vet_typ1', 'uc_yn', 'int_yn', 'ptot_r', 'paw_yn', 'earner', 'hfdval']
    NIU = ['ssikidyn', 'resnssi1', 'resnssi2', 'pedisdrs', 'pedisear', 'pediseye', 'pedisout', 'pedisphy', 'pedisrem', 'pemlr', 'oi_off']
    # Add variables that indicate which entries of which columns contain missing values
    for col in Noneornot:
        CPS[col + '_missing_NoneIn'] = np.where(CPS[col] == 'None or not in universe', 1, 0)
    for col in NotInUniv:
        CPS[col + '_missing_NotIn'] = np.where(CPS[col] == 'Not in universe', 1, 0)
    for col in NIU:
        CPS[col + '_missing_NIU'] = np.where(CPS[col] == 'NIU', 1, 0)
    
    # Setting missing values equal to zero after adding binary variable
    CPS = CPS.replace({'None or not in universe' : 0.}, regex = True)
    CPS = CPS.replace({'Not in universe' : 0.}, regex = True)
    CPS = CPS.replace({'NIU' : 0.}, regex = True)
    CPS = CPS.replace({'Did not receive SSI' : 0.}, regex = True)
    CPS = CPS.replace({'Received SSI' : 1.}, regex = True)
    CPS = CPS.replace({'No' : 0.}, regex = True)
    # Creating unearned income variable
    CPS_unearned = (CPS[['fssval' ,'fretval' ,'foival' , 'fucval' , 'fintval', 'frntval', 'fdivval', 'fvetval', 'fcspval']].astype(float)).copy()
    unearned_income = CPS_unearned.sum(axis = 1)
    # Creating earned income variable
    CPS_earned = (CPS[['fwsval' ,'fseval','ffrval']].astype(float)).copy()
    earned_income = CPS_earned.sum(axis = 1)
    # Determining current recipients of SSI benefit
    CPS['current_recipient'] = np.where((CPS.ssi_yn =='Yes'), 1, 0)
    CPS = CPS.replace({'Yes' : 1.}, regex = True)

    CPS['fssival'] = (CPS['fssival'].astype(float)).copy()
    CPS['earned_income'] = earned_income
    CPS['unearned_income'] = unearned_income
    # Below is a list of variables that are categorical but represented as strings
    cols = ['a_ftpt', 'filestat', 'peridnum', 'ptot_r','ftot_r', 'pemlr', 'resnssi1', 'resnssi2', 'a_maritl', 'fownu18', 'oi_off', 'rsnnotw', 'pedisdrs', 'earner', 'prdtrace','hea', 'earner']
    for col in cols:#Iterating through all of these categorical strings and converting to numbers below
        CPS[col] = CPS[col].astype('category')

    cat_columns = CPS.select_dtypes(['category']).columns
    CPS[cat_columns] = CPS[cat_columns].apply(lambda x: x.cat.codes)

    return CPS

use_spm_data = False

CPS = pd.read_csv('../cpsmar2014t.csv')
CPS = Init_CPS_Vars(CPS)

CPS.a_age = np.where(CPS.a_age == "80-84 years of age",
                     np.mean(np.arange(80,85)),
                     CPS.a_age)
CPS.a_age = np.where(CPS.a_age == "85+ years of age",
                     np.mean(np.arange(85,96)),
                     CPS.a_age)
CPS.a_age = pd.to_numeric(CPS.a_age)
CPS['80_84__missing'] = np.where(CPS['a_age'] == np.mean(np.arange(80,85)), 1, 0)
CPS['85_95__missing'] = np.where(CPS['a_age'] == np.mean(np.arange(85,86)), 1, 0)

key1 = CPS.columns.to_series().groupby(CPS.dtypes).groups.keys()[1]
cols =  CPS.columns.to_series().groupby(CPS.dtypes).groups[key1].values
for col in cols:
    CPS[col] = CPS[col].astype('category')
CPS[cols] = CPS[cols].apply(lambda x: x.cat.codes)
CPS = CPS.fillna(0) 

CPS_unearned = (CPS[['fssval' ,'fretval' ,'foival' , 'fucval' , 'fintval', 'frntval', 'fdivval', 'fvetval', 'fcspval']].astype(float)).copy()
unearned_income = CPS_unearned.sum(axis = 1)
# Creating earned income variable
CPS_earned = (CPS[['fwsval' ,'fseval','ffrval']].astype(float)).copy()
earned_income = CPS_earned.sum(axis = 1)
# Determining current recipients of SSI benefit

CPS['fam_earned_income'] = earned_income
CPS['fam_unearned_income'] = unearned_income
CPS.to_pickle('CPS_family.pickle')


# CPS_dataset = pd.read_pickle('CPS_family.pickle')
CPS_dataset = CPS.copy()

/Users/chrisrytting1/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (5,23,24,29,83,265,282) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
if use_spm_data == True:
    # Get data from https://www.census.gov/data/datasets/2013/demo/supplemental-poverty-measure/spm.html SPM data from 2013 since 2014 cps reflects what happened in 2013
    spm_data = pd.read_stata('spmresearch2013new.dta')
    CPS_dataset = CPS_dataset.merge(spm_data[['spmu_caphousesub', 'h_seq', 'pppos']], on = ['h_seq', 'pppos'])
    CPS_dataset['fhoussub'] = CPS_dataset['spmu_caphousesub']
    CPS_dataset = CPS_dataset.drop('spmu_caphousesub', 1)


#Removing those who receive hlorent, but who have no subsequent fhoussub value to describe it
CPS_dataset.hlorent = np.where((CPS_dataset.hlorent == 1) & (CPS_dataset.fhoussub == 0),
     0, CPS_dataset.hlorent)
CPS_dataset.hpublic = np.where((CPS_dataset.hpublic == 1) & (CPS_dataset.fhoussub == 0),
     0, CPS_dataset.hpublic)
CPS_dataset['housing'] = np.where((CPS_dataset.hpublic == 1) | (CPS_dataset.hlorent == 1),
     1, 0)
CPS_dataset = CPS_dataset.drop(['hpublic', 'hlorent'],1)

#Earned income
p_earned = CPS_dataset.wsal_val + CPS_dataset.semp_val + CPS_dataset.frse_val #individual earned income
CPS_dataset['p_earned'] = p_earned


#Unearned income
p_unearned = CPS_dataset.ss_val + CPS_dataset.rtm_val + CPS_dataset.div_val + \
    CPS_dataset.oi_off + CPS_dataset.uc_yn + CPS_dataset.int_yn
CPS_dataset['p_unearned'] = p_unearned

#Net Income
CPS_dataset['family_net_income'] = p_earned + p_unearned

#disabled (check reg if categorical or binary is better after the sum)
CPS_dataset['disability'] = np.zeros(len(CPS_dataset))
CPS_dataset.disability = np.where(CPS_dataset.pedisdrs == 1, 1, CPS_dataset.disability)
CPS_dataset.disability = np.where(CPS_dataset.pedisear == 1, 1, CPS_dataset.disability)
CPS_dataset.disability = np.where(CPS_dataset.pediseye == 1, 1, CPS_dataset.disability)
CPS_dataset.disability = np.where(CPS_dataset.pedisout == 1, 1, CPS_dataset.disability)
CPS_dataset.disability = np.where(CPS_dataset.pedisphy == 1, 1, CPS_dataset.disability)
CPS_dataset.disability = np.where(CPS_dataset.pedisrem == 1, 1, CPS_dataset.disability)



# #State residency of family and weights

CPS_dataset_mean = CPS_dataset.groupby(['fh_seq', 'ffpos']).mean()
CPS_dataset_sum = CPS_dataset.groupby(['fh_seq', 'ffpos']).sum()
CPS_dataset2 = pd.merge(CPS_dataset_mean, CPS_dataset_sum, left_index = True, right_index = True, how = 'inner',\
                       suffixes = ('_mean', '_sum'))

CPS = CPS_dataset2
# CPS_dataset2.to_pickle('use_df_hlo_rf.pickle')

In [5]:
# CPS = pd.read_pickle('use_df_hlo_rf.pickle')
Rf = rf(n_estimators = 200) # Creating Random Forest 
CPS_use = CPS.drop('peridnum_mean', 1)
CPS_use = CPS.drop('peridnum_sum', 1)

#Splitting data into training and test sets
train = CPS_use.sample(frac=0.8, random_state=1)
train_x = train.copy()
train_x = train_x.drop(['hunits_sum', 'hhpos_sum', 'h_seq_sum', 'hrecord_sum', 'ph_seq_sum',\
                        'housing_sum','fhoussub_sum', 'hssival_sum', 'hunits_mean', 'hhpos_mean',\
                        'h_seq_mean', 'hrecord_mean', 'ph_seq_mean','housing_mean','fhoussub_mean',\
                        'hssival_mean', 'hsup_wgt_mean', 'fsup_wgt_mean', 'marsupwt_mean',\
                        'hsup_wgt_sum', 'marsupwt_sum', 'fsup_wgt_sum','htotval_mean', 'peridnum_mean',\
                        'h_idnum1_sum', 'h_idnum1_mean','htotval_sum','hfdval_sum'],1)



test_x = CPS_use.loc[~CPS_use.index.isin(train_x.index)]
test_y = test_x['housing_mean']
test_x = test_x.drop(['hunits_sum', 'hhpos_sum', 'h_seq_sum', 'hrecord_sum', 'ph_seq_sum',\
                        'housing_sum','fhoussub_sum', 'hssival_sum', 'hunits_mean', 'hhpos_mean',\
                        'h_seq_mean', 'hrecord_mean', 'ph_seq_mean','housing_mean','fhoussub_mean',\
                        'hssival_mean', 'hsup_wgt_mean', 'fsup_wgt_mean', 'marsupwt_mean',\
                        'hsup_wgt_sum', 'marsupwt_sum', 'fsup_wgt_sum','htotval_mean', 'peridnum_mean',\
                        'h_idnum1_sum', 'h_idnum1_mean','htotval_sum','hfdval_sum'],1)


# Fitting the Random Forest model to the training set
Rf.fit(train_x, train['housing_mean'])
print Rf.feature_importances_, train_x.columns.values
predictions = Rf.predict(test_x)
print 'score:', Rf.score(test_x, test_y) # Printing Random Forest accuracy
fimp = Rf.feature_importances_
colvals = train_x.columns.values

[ 0.00348351  0.00105306  0.         ...,  0.00204017  0.00282958
  0.00081407] ['hefaminc_mean' 'h_respnm_mean' 'h_year_mean' ..., 'p_unearned_sum'
 'family_net_income_sum' 'disability_sum']
score: 0.965046604527


In [6]:
features = Rf.feature_importances_
maxes = np.argsort(features)[::-1]
print maxes
print list(train_x[maxes].columns.values)
print features[maxes]

[ 848   12  186 ...,  116 1219 1217]
['housret_sum', 'h_tenure_mean', 'f_mv_fs_mean', 'housret_mean', 'f_mv_fs_sum', 'hprop_val_sum', 'hprop_val_mean', 'prop_tax_sum', 'ffngcaid_sum', 'ffngcaid_mean', 'prop_tax_mean', 'hfoodno_mean', 'hfdval_mean', 'hfdval_missing_NotIn_mean', 'hpctcut_mean', 'ffngcare_mean', 'hfoodsp_sum', 'ffngcare_sum', 'p_mvcaid_mean', 'h_tenure_sum', 'hfoodsp_mean', 'a_age_mean', 'hfoodmo_mean', 'hfoodmo_sum', 'ftotval_mean', 'ptotval_mean', 'a_age_sum', 'hothval_mean', 'ptotval_sum', 'gtcbsa_mean', 'gestfips_mean', 'ftotval_sum', 'fmoop_mean', 'p_mvcaid_sum', 'gestfips_sum', 'hefaminc_mean', 'pothval_mean', 'fothval_mean', 'gtcbsa_sum', 'a_fnlwgt_sum', 'mon_mean', 'moop_mean', 'hearnval_mean', 'gestcen_mean', 'hfoodno_sum', 'hearnval_sum', 'fmoop_sum', 'hfdval_missing_NotIn_sum', 'a_fnlwgt_mean', 'i_hloren_sum', 'hefaminc_sum', 'moop_sum', 'pothval_sum', 'hhinc_mean', 'povll_sum', 'age1_mean', 'hengval_mean', 'gestcen_sum', 'prdtrace_mean', 'hhinc_sum', 'a_hga_me

In [7]:
get_probs = CPS_use.copy()
get_probs = get_probs.drop(['hunits_sum', 'hhpos_sum', 'h_seq_sum', 'hrecord_sum', 'ph_seq_sum',\
                        'housing_sum','fhoussub_sum', 'hssival_sum', 'hunits_mean', 'hhpos_mean',\
                        'h_seq_mean', 'hrecord_mean', 'ph_seq_mean','housing_mean','fhoussub_mean',\
                        'hssival_mean', 'hsup_wgt_mean', 'fsup_wgt_mean', 'marsupwt_mean',\
                        'hsup_wgt_sum', 'marsupwt_sum', 'fsup_wgt_sum','htotval_mean', 'peridnum_mean',\
                        'h_idnum1_sum', 'h_idnum1_mean','htotval_sum','hfdval_sum'],1)

prediction_vec = Rf.predict_proba(get_probs)
print prediction_vec

[[ 1.     0.   ]
 [ 1.     0.   ]
 [ 1.     0.   ]
 ..., 
 [ 1.     0.   ]
 [ 1.     0.   ]
 [ 0.995  0.005]]


In [8]:
if use_spm_data == True:
    np.savetxt('rf_probs_spm.csv', prediction_vec)
    
else:
    np.savetxt('rf_probs.csv', prediction_vec)